# Sklearn 
    Bailey Smith
    February 15 2018

In [1]:
import numpy as np
import pandas as pd
from numpy import linalg as la
import statsmodels.formula.api as smf
from sklearn.pipeline import Pipeline
from sklearn import datasets, svm, decomposition
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator, ClassifierMixin, TransformerMixin 
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV

## Implementation of GDA matching `sklearn` conventions.

In [2]:
class GDA(BaseEstimator, ClassifierMixin):
    '''Gaussian Discriminant Analysis Algorithm'''
    
    def __init__(self):
        """Initialize system parameters."""
        
    
    def fit(self, X, y=None):
        '''Train on the data given - store pi, mu, and sigma for each class
        Parameters:
            X - array of training data
            y - array of train targets (Optional)
        '''
        self.pis_ = []
        self.mus_ = []
        self.sigmas_ = []
        self.classes_ = []
        self.inv_sig_ = []
        self.det_sig_ = []
#         self.y_pred_ = 0
        
        self.classes_ = list(set(y))
        N = len(self.classes_)
        i = 0
        for class_ in self.classes_:
            xs = X[y == class_]
            self.pis_.append(len(y[y == class_])/N)
            self.mus_.append(np.mean(xs,axis=0))
            self.sigmas_.append(np.cov(xs.T))
            
            # Save the inverse and determinant of each sigma
            self.inv_sig_.append(la.inv(self.sigmas_[i]))
            self.det_sig_.append(la.det(2*np.pi*self.sigmas_[i]))
            i += 1

        # Save each list as an array
        self.pis_ = np.array(self.pis_)
        self.mus_ = np.array(self.mus_)
        self.sigmas_ = np.array(self.sigmas_)
        self.inv_sig_ = np.array(self.inv_sig_)
        self.det_sig_ = np.array(self.det_sig_)
        return self
        
    def predict_proba(self, X):
        '''Given new data, find the probability each data point is in each class
        Parameters:
            X - array (test data)
        Returns:
            total_prob - array of probabilities'''
        
        total_prob = []
        # Calculate probabilities for each data point
        for x in X:
            prob = []
            for i in range(len(self.classes_)):
                # Numerator of p(y = c|x)
                prob.append(self.pis_[i] * self.det_sig_[i]**(-.5) * 
                            np.exp(-.5 * (x - self.mus_[i]).T.dot(self.inv_sig_[i]).dot(x - self.mus_[i])))
            
            total_prob.append(np.array(prob)/np.sum(prob))
            
        return np.array(total_prob)
        
    def predict(self, X):
        '''Predict the class for each data point
        Parameters:
            X - array (test data)
        Returns:
            array of predictions
        '''
        predictions = []
        # Get all probabilities
        total_prob = self.predict_proba(X)
        for probs in total_prob:
            # Find the largest probability
            predictions.append(self.classes_[np.argmax(probs)])
        
        self.y_pred_ = np.array(predictions)
        return self.y_pred_

In [4]:
iris = datasets.load_iris()

X_train, X_test, Y_train, Y_test = train_test_split(iris.data,iris.target,test_size=0.3)

gda = GDA()
gda.fit(X_train, Y_train)
gda.predict(X_test)

score = gda.score(X_test, Y_test)
print("Accuracy:", score)

Accuracy: 0.977777777778


A transformer class where the `fit()` and `transform()` methods takes in $X$ as a pandas Data Frame.
- For each numerical column, replaces any `nan` entries with the mean of the column.
- Drops string columns.
- Returns the data as a NumPy array.

In [5]:
class NormalizingTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.mean = X.mean(numeric_only=True)
        return self

    def transform(self, X):
        """"""
        X = X.fillna(self.mean)
        X = X.select_dtypes(exclude=['object'])
        return X.as_matrix()

Used `cross_validate()` to score GDA class on the iris dataset.
Did the same for a `LogisticRegressionClassifier`.

In [6]:
cross_validate(gda, iris.data, iris.target, cv=4, return_train_score=True)

{'fit_time': array([ 0.00120401,  0.00072718,  0.00613403,  0.00130177]),
 'score_time': array([ 0.00294399,  0.00263977,  0.00458002,  0.00202608]),
 'test_score': array([ 1.        ,  0.94871795,  0.94444444,  1.        ]),
 'train_score': array([ 0.97297297,  0.99099099,  0.99122807,  0.98245614])}

In [7]:
logistic = LogisticRegression()
cross_validate(gda, iris.data, iris.target, cv=4, return_train_score=True)

{'fit_time': array([ 0.00162411,  0.00067568,  0.0030632 ,  0.00122499]),
 'score_time': array([ 0.00353193,  0.00504112,  0.00255275,  0.00358701]),
 'test_score': array([ 1.        ,  0.94871795,  0.94444444,  1.        ]),
 'train_score': array([ 0.97297297,  0.99099099,  0.99122807,  0.98245614])}

Used the cancer data set (`datasets.load_breast_cancer()`) and did a grid search on an SVM (`sklearn.linear.svm`) with the parameter `C` as .01, .1, or 1, and the parameter `kernel` as `"linear"`, `"poly"`, `"rbf"`, and `"sigmoid"`.
Answers the following questions:
- What is the best choice of parameters?
- How well does the corresponding model do?

In [8]:
cancer = datasets.load_breast_cancer()
data = cancer.data

# Normalize data using preprocessing
data = StandardScaler().fit_transform(data)

Cs = [.01,.1,1]
kernels = ['linear', 'poly', 'rbf', 'sigmoid']
parameters = {'C':Cs, 'kernel':kernels}
est = svm.SVC(max_iter=1000)

clf = GridSearchCV(estimator=est, param_grid=parameters, n_jobs=-1)
clf.fit(data, cancer.target) 
score = clf.best_score_
params = clf.best_params_

print(f"Best parameters: {params}\nScore of corresponding model: {score}")

/Users/baileysmith/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/baileysmith/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/baileysmith/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


Best parameters: {'C': 0.1, 'kernel': 'linear'}
Score of corresponding model: 0.9736379613356766


Made a pipeline of the previously made transformer, a normalizing scaler transformer (`preprocessing.StandardScaler`), a PCA transformer (`decomposition.PCA`), and an SVM classifier (`svm.SVC`).
Using the titanic dataset, did a grid search for the best model, varying the parameters.

Answers the questions:
- What is your best choice of parameters?
- How well does the corresponding model do?

In [48]:
# Import dataset.
titanic = pd.read_csv("titanic.csv")

#Drop columns that don't matter
titanic = titanic.drop(['Name','Sibsp', 'Parch','Boat', 'Body', 'Cabin', 'Ticket', 'home.dest'], axis=1)

# Fill missing values
titanic = titanic.fillna(titanic.mean())

titanic.Pclass = titanic.Pclass.astype(int)
titanic.Survived = titanic.Survived.astype(int)

# Create dummy variables for city of departure, gender and passenger class
titanic = pd.get_dummies(titanic, columns=['Embarked'], drop_first=True) 
titanic = pd.get_dummies(titanic, columns=['Sex'], drop_first=True)
titanic = pd.get_dummies(titanic, columns=['Pclass'], drop_first=True)

# Make dependent variable
y_data = titanic.Survived.as_matrix()
X_data = titanic[['Embarked_Q','Embarked_S', 'Sex_male', 'Age', 'Fare', 'Pclass_2', 'Pclass_3']]
# X_data = StandardScaler().fit_transform(X_data)

In [52]:
NT = NormalizingTransformer()
nst = StandardScaler()
pca = decomposition.PCA()
svc = svm.SVC(max_iter=1000)
pipe = Pipeline(steps=[('normalizing transformer', NT),('standard scalar', nst),('pca', pca),('svc', svc)])

n_components = [1, 3, 6]
ks = np.linspace(-10,10,21)
Cs = 10**ks
kernels = ['linear', 'poly', 'rbf', 'sigmoid']
estimator = GridSearchCV(pipe,dict(pca__n_components=n_components,svc__C=Cs, svc__kernel=kernels))

estimator.fit(X_data, y_data)

/Users/baileysmith/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/baileysmith/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/baileysmith/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/baileysmith/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or M

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('normalizing transformer', NormalizingTransformer()), ('standard scalar', StandardScaler(copy=True, with_mean=True, with_std=True)), ('pca', PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('svc', SVC(C=1.0, cache_size...  max_iter=1000, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'pca__n_components': [1, 3, 6], 'svc__C': array([  1.00000e-10,   1.00000e-09,   1.00000e-08,   1.00000e-07,
         1.00000e-06,   1.00000e-05,   1.00000e-04,   1.00000e-03,
         1.00000e-02,   1.00000e-01,   1.00000e+00,   1.00000e+01,
         1.00000e+02,   1.00000e+03,   1.00000e+04,   1.00000e+05,
         1.00000e+06,   1.00000e+07,   1.00000e+08,   1.00000e+09,
         1.00000e+10]), 'svc__kernel': ['linear', 'poly', 'rbf', 's

In [53]:
score = estimator.best_score_
params = estimator.best_params_

print(f"Best parameters: {params}\nScore of corresponding model: {score}")

Best parameters: {'pca__n_components': 6, 'svc__C': 0.10000000000000001, 'svc__kernel': 'linear'}
Score of corresponding model: 0.6572519083969466
